In [5]:
import pandas as pd
import numpy as np
import torch
import re
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import os

In [6]:
SRC = 'PDF1'
FROM_ID = 501
TO_ID = 625
# INPUT_FILE = f"../data/preprocessing_data/{SRC.lower()}_{FROM_ID}_{TO_ID}.csv"
INPUT_FILE = f"../output/ocr/{SRC}_{FROM_ID}_{TO_ID}.csv"
OUTPUT_FILE = f"../output/alignment/{SRC}_{FROM_ID}_{TO_ID}.csv"
ALIGN_THRESHOLD = 0.75
MIN_SENTENCE_LENGTH = 5

In [7]:
def get_device():
    if torch.cuda.is_available():
        return 'cuda'
    return 'cpu'

def clean_text(text):
    if not isinstance(text, str): return ""
    text = re.sub(r'^[^:\n]+:\s*', '', text) # Bỏ tên người nói
    text = text.replace('\n', ' ').replace('\t', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def split_sentences(text, lang='vi'):
    text = clean_text(text)
    if not text: return []
    if lang == 'zh': 
        sents = re.split(r'(?<=[。！？])\s*', text)
    else: 
        sents = re.split(r'(?<=[.?!])\s+(?=[A-Z"\'(])', text)
    return [s.strip() for s in sents if len(s.strip()) >= MIN_SENTENCE_LENGTH]

def align_n3(src_sents, tgt_sents, model, device, threshold):
    if not src_sents or not tgt_sents:
        return []

    # Encode
    # 1-gram (Câu đơn)
    emb_src = model.encode(src_sents, convert_to_tensor=True, device=device)
    emb_tgt = model.encode(tgt_sents, convert_to_tensor=True, device=device)
    
    # 2-gram (Gộp 2 câu)
    tgt_merged_2 = [tgt_sents[i] + " " + tgt_sents[i+1] for i in range(len(tgt_sents)-1)]
    src_merged_2 = [src_sents[i] + src_sents[i+1] for i in range(len(src_sents)-1)]
    
    emb_tgt_2 = model.encode(tgt_merged_2, convert_to_tensor=True, device=device) if tgt_merged_2 else None
    emb_src_2 = model.encode(src_merged_2, convert_to_tensor=True, device=device) if src_merged_2 else None

    # 3-gram (Gộp 3 câu) 
    tgt_merged_3 = [tgt_sents[i] + " " + tgt_sents[i+1] + " " + tgt_sents[i+2] for i in range(len(tgt_sents)-2)]
    src_merged_3 = [src_sents[i] + src_sents[i+1] + src_sents[i+2] for i in range(len(src_sents)-2)]
    
    emb_tgt_3 = model.encode(tgt_merged_3, convert_to_tensor=True, device=device) if tgt_merged_3 else None
    emb_src_3 = model.encode(src_merged_3, convert_to_tensor=True, device=device) if src_merged_3 else None

    # Tính ma trận tương đồng (similarity matrices)
    # Khởi tạo ma trận (rỗng chiều)
    def get_sim(emb1, emb2):
        if emb1 is not None and emb2 is not None:
            return util.cos_sim(emb1, emb2).cpu().numpy()
        return np.zeros((0,0))

    sim_1_1 = get_sim(emb_src, emb_tgt)
    
    sim_1_2 = get_sim(emb_src, emb_tgt_2)     # Src đơn vs Tgt gộp 2
    sim_2_1 = get_sim(emb_src_2, emb_tgt)     # Src gộp 2 vs Tgt đơn
    
    sim_1_3 = get_sim(emb_src, emb_tgt_3)     # Src đơn vs Tgt gộp 3 
    sim_3_1 = get_sim(emb_src_3, emb_tgt)     # Src gộp 3 vs Tgt đơn

    results = []
    
    # Đánh dấu câu đã dùng
    used_src = np.zeros(len(src_sents), dtype=bool)
    used_tgt = np.zeros(len(tgt_sents), dtype=bool)

    # Greedy search
    while True:
        best_score = -1
        best_type = None
        best_indices = None # (r, c)

        # Helper để check max score trong matrix
        def check_matrix(matrix, label):
            nonlocal best_score, best_type, best_indices
            if matrix.size > 0:
                r, c = np.unravel_index(matrix.argmax(), matrix.shape)
                if matrix[r, c] > best_score:
                    best_score = matrix[r, c]
                    best_type = label
                    best_indices = (r, c)

        # Kiểm tra tất cả các trường hợp
        check_matrix(sim_1_1, '1-1')
        check_matrix(sim_1_2, '1-2')
        check_matrix(sim_2_1, '2-1')
        check_matrix(sim_1_3, '1-3') 
        check_matrix(sim_3_1, '3-1') 

        if best_score < threshold:
            break

        r, c = best_indices
        valid_match = False
        
        # Logic kết quả và masking
        # Logic chung: Kiểm tra used -> Add result -> Mark used -> Mask matrix
        
        if best_type == '1-1':
            if not used_src[r] and not used_tgt[c]:
                results.append({'src': src_sents[r], 'tgt': tgt_sents[c], 'type': '1-1'})
                used_src[r] = True; used_tgt[c] = True
                valid_match = True
                # Masking: Xóa hàng r (src) và cột c (tgt) ở mọi matrix liên quan: gán -1 trực tiếp vào các vùng ảnh hưởng
                sim_1_1[r, :] = -1; sim_1_1[:, c] = -1
                if sim_1_2.size: sim_1_2[r, :] = -1; sim_1_2[:, max(0, c-1):c+1] = -1
                if sim_2_1.size: sim_2_1[max(0, r-1):r+1, :] = -1; sim_2_1[:, c] = -1
                if sim_1_3.size: sim_1_3[r, :] = -1; sim_1_3[:, max(0, c-2):c+1] = -1
                if sim_3_1.size: sim_3_1[max(0, r-2):r+1, :] = -1; sim_3_1[:, c] = -1

        elif best_type == '1-2': # Src r vs Tgt c, c+1
            if not used_src[r] and not used_tgt[c] and not used_tgt[c+1]:
                results.append({'src': src_sents[r], 'tgt': tgt_sents[c] + " " + tgt_sents[c+1], 'type': '1-2'})
                used_src[r] = True; used_tgt[c:c+2] = True
                valid_match = True
                sim_1_2[r, c] = -1 # Xóa chính nó
                sim_1_1[r, :] = -1; sim_1_1[:, c:c+2] = -1
               

        elif best_type == '2-1': # Src r, r+1 vs Tgt c
            if not used_src[r] and not used_src[r+1] and not used_tgt[c]:
                results.append({'src': src_sents[r]+src_sents[r+1], 'tgt': tgt_sents[c], 'type': '2-1'})
                used_src[r:r+2] = True; used_tgt[c] = True
                valid_match = True
                sim_2_1[r, c] = -1
                sim_1_1[r:r+2, :] = -1; sim_1_1[:, c] = -1

        elif best_type == '1-3': # Src r vs Tgt c, c+1, c+2
            if not used_src[r] and not used_tgt[c] and not used_tgt[c+1] and not used_tgt[c+2]:
                results.append({'src': src_sents[r], 'tgt': tgt_sents[c]+" "+tgt_sents[c+1]+" "+tgt_sents[c+2], 'type': '1-3'})
                used_src[r] = True; used_tgt[c:c+3] = True
                valid_match = True
                sim_1_3[r, c] = -1
                sim_1_1[r, :] = -1; sim_1_1[:, c:c+3] = -1

        elif best_type == '3-1': # Src r, r+1, r+2 vs Tgt c
            if not used_src[r] and not used_src[r+1] and not used_src[r+2] and not used_tgt[c]:
                results.append({'src': src_sents[r]+src_sents[r+1]+src_sents[r+2], 'tgt': tgt_sents[c], 'type': '3-1'})
                used_src[r:r+3] = True; used_tgt[c] = True
                valid_match = True
                sim_3_1[r, c] = -1
                sim_1_1[r:r+3, :] = -1; sim_1_1[:, c] = -1

        # Nếu không match được (do conflict đã bị dùng) -> xóa điểm này để tìm max tiếp theo
        if not valid_match:
            if best_type == '1-1': sim_1_1[r, c] = -1
            elif best_type == '1-2': sim_1_2[r, c] = -1
            elif best_type == '2-1': sim_2_1[r, c] = -1
            elif best_type == '1-3': sim_1_3[r, c] = -1
            elif best_type == '3-1': sim_3_1[r, c] = -1

        # Cập nhật kết quả
        if valid_match:
            results[-1]['score'] = float(best_score)

    return results

In [8]:
device = get_device()
print(f"Đang tải model LaBSE trên {device} (n=3)...")
model = SentenceTransformer('sentence-transformers/LaBSE', device=device)

df = pd.read_csv(INPUT_FILE)
final_data = []

print("Bắt đầu Alignment (1-1, 1-2, 2-1, 1-3, 3-1)...")
for idx, row in tqdm(df.iterrows(), total=len(df)):
    src_sents = split_sentences(row.get('src_lang', ''), lang='zh')
    tgt_sents = split_sentences(row.get('tgt_lang', ''), lang='vi')
    
    pairs = align_n3(src_sents, tgt_sents, model, device, ALIGN_THRESHOLD)
    
    for p in pairs:
        final_data.append({
            'src_id': row.get('src_id', ''),
            'src_lang': p['src'],
            'tgt_lang': p['tgt'],
            'score': p['score'],
            'type': p.get('type', '1-1')
        })

# Thống kê báo cáo
df_res = pd.DataFrame(final_data)
type_counts = df_res['type'].value_counts()

print("Kết quả Alignment")
print(f"Tổng số cặp: {len(df_res)}")
print(f"Phân bố cặp câu: {type_counts}")
print(f"Average Semantic Score: {df_res['score'].mean():.4f}")

# Lưu file
df_res[['src_id', 'src_lang', 'tgt_lang']].to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
print(f"Đã lưu file: {OUTPUT_FILE}")

Đang tải model LaBSE trên cuda (n=3)...
Bắt đầu Alignment (1-1, 1-2, 2-1, 1-3, 3-1)...


100%|██████████| 115/115 [00:04<00:00, 27.31it/s]

Kết quả Alignment
Tổng số cặp: 125
Phân bố cặp câu: type
1-1    86
1-2    23
2-1     8
1-3     7
3-1     1
Name: count, dtype: int64
Average Semantic Score: 0.8053
Đã lưu file: ../output/alignment/PDF1_501_625.csv
